In [112]:
import pandas as pd
import numpy as np
import csv

In [2]:
# define file name
data_folder_path = "event-recommendation-engine-challenge"
event_attendees_file_name = data_folder_path + "/event_attendees.csv"
events_file_name = data_folder_path + "/events.csv"
test_file_name = data_folder_path + "/test.csv"
train_file_name = data_folder_path + "/train.csv"
users_file_name = data_folder_path + "/users.csv"
user_friends_file_name = data_folder_path + "/user_friends.csv"

In [3]:
# read data
print("Reading files ...")
try:
    print(event_attendees_file_name + " ... ", end = "")
    event_attendees = pd.read_csv(event_attendees_file_name)
    print("finished (1/6)")
    print(events_file_name + " ... ", end = "")
    events = pd.read_csv(events_file_name)
    print("finished (2/6)")
    print(test_file_name + " ... ", end = "")
    test = pd.read_csv(test_file_name)
    print("finished (3/6)")
    print(train_file_name + " ... ", end = "")
    train = pd.read_csv(train_file_name)
    print("finished (4/6)")
    print(users_file_name + " ... ", end = "")
    users = pd.read_csv(users_file_name)
    print("finished (5/6)")
    print(user_friends_file_name + " ... ", end = "")
    user_friends = pd.read_csv(user_friends_file_name)
    print("finished (6/6)")
    print("File loading completed!")
except FileNotFoundError as fnf_error:
    print(fnf_error)

Reading files ...
event-recommendation-engine-challenge/event_attendees.csv ... finished (1/6)
event-recommendation-engine-challenge/events.csv ... finished (2/6)
event-recommendation-engine-challenge/test.csv ... finished (3/6)
event-recommendation-engine-challenge/train.csv ... finished (4/6)
event-recommendation-engine-challenge/users.csv ... finished (5/6)
event-recommendation-engine-challenge/user_friends.csv ... finished (6/6)
File loading completed!


In [4]:
# check shape of all data
print(event_attendees.shape)
print(events.shape)
print(test.shape)
print(train.shape)
print(users.shape)
print(user_friends.shape)

(24144, 5)
(3137972, 110)
(10237, 4)
(15398, 6)
(38209, 7)
(38202, 2)


In [5]:
# check event_attendees examples
event_attendees.iloc[0 : 10,]

,event,yes,maybe,invited,no
0,1159822043,1975964455 252302513 4226086795 3805886383 142...,2733420590 517546982 1350834692 532087573 5831...,1723091036 3795873583 4109144917 3560622906 31...,3575574655 1077296663
1,686467261,2394228942 2686116898 1056558062 3792942231 41...,1498184352 645689144 3770076778 331335845 4239...,1788073374 733302094 1830571649 676508092 7081...,NaN
2,1186208412,NaN,3320380166 3810793697,1379121209 440668682,1728988561 2950720854
3,2621578336,NaN,NaN,NaN,NaN
4,855842686,2406118796 3550897984 294255260 1125817077 109...,2671721559 1761448345 2356975806 2666669465 10...,1518670705 880919237 2326414227 2673818347 332...,3500235232
5,2018671985,NaN,NaN,NaN,NaN
6,488116622,4145960786 2550625355 2577667841 1575121941 28...,1227223575 2789471603 1323321680 3086272918 38...,1413359297 2300232602 1412759254 617751520 286...,1498160155 3708150269 823488244 3595018395 173...
7,1273761447,2680366192 2151335654 3447231284 3021641283 17...,94519567 2208454642 1749189642 2558652483 2983...,3828226993 3279641599 493535713 2091128996 298...,3215817616 101410505 3172228763 4161238910 181...
8,2688888297,298428624 2292079981 1819927116 1843127538 410...,3433056329 3166442492 1754661408 2966742619 32...,143382439 552645572 2872499486 1476024415 3890...,3433837562 492244978 784111553 3319042922
9,3870329460,NaN,4229976635,101440046 3547967849 2482041922 662878699 2600...,1200610016 379229947 1357977256 725446989


In [6]:
# create event_attendees_dict & user_interests_dict
# "1" = yes, "2" = maybe, "3" = "invited", "4" = no
# event_attendees_dict = {event_id : {"1" : [], "2" : [], "3" : [], "4" : []}}
# user_interests_dict = {user_id : {"1" : [], "2" : [], "3" : [], "4" : []}}
def create_event_attendees_dict_and_user_interests_dict(event_attendees):
    event_attendees_dict = {}
    user_interests_dict = {}
    for index, row in event_attendees.iterrows():
        event_dict = {}
        event_id = str(row[0])
        for i in range(1, len(row)):
            l = row[i]
            if str(l) == "nan":
                event_dict[str(i)] = []
            else:
                user_id_list = l.split(" ")
                event_dict[str(i)] = user_id_list
                for user_id in user_id_list:
                    if not user_id in user_interests_dict:
                        user_interests_dict[user_id] = {}
                    if str(i) in user_interests_dict[user_id]:
                        user_interests_dict[user_id][str(i)].append(event_id)
                    else:
                        user_interests_dict[user_id][str(i)] = [event_id]
        event_attendees_dict[event_id] = event_dict
    return [event_attendees_dict, user_interests_dict]

In [7]:
train.iloc[0 : 10,]

,user,event,invited,timestamp,interested,not_interested
0,3044012,1918771225,0,2012-10-02 15:53:05.754000+00:00,0,0
1,3044012,1502284248,0,2012-10-02 15:53:05.754000+00:00,0,0
2,3044012,2529072432,0,2012-10-02 15:53:05.754000+00:00,1,0
3,3044012,3072478280,0,2012-10-02 15:53:05.754000+00:00,0,0
4,3044012,1390707377,0,2012-10-02 15:53:05.754000+00:00,0,0
5,3044012,1532377761,0,2012-10-02 15:53:05.754000+00:00,0,0
6,4236494,2352676247,0,2012-10-30 01:48:25.617000+00:00,0,0
7,4236494,152418051,0,2012-10-30 01:48:28.645000+00:00,1,0
8,4236494,4203627753,0,2012-10-30 01:49:14.152000+00:00,1,0
9,4236494,110357109,0,2012-10-30 01:48:25.617000+00:00,0,0


In [147]:
# train_dict = {user_id : {event_id : [invited, interested, not_interested]}}
def create_train_dict(train):
    duplicated_num = 0
    multiple_num = 0
    train_dict = {}
    for index, row in train.iterrows():
        user_id = str(row[0])
        event_id = str(row[1])
        invited = row[2]
        interested = row[4]
        not_interested = row[5]
        if not user_id in train_dict:
            train_dict[user_id] = {}
        if event_id in train_dict[user_id]:
            if train_dict[user_id][event_id] == [invited, interested, not_interested]:
                duplicated_num += 1
            else:
                multiple_num += 1
        else:
            train_dict[user_id][event_id] = [invited, interested, not_interested]
    print("Duplicated data in user: " + str(duplicated_num))
    print("Multiple data in user: " + str(multiple_num))
    return train_dict

In [9]:
test.iloc[0 : 10,]

,user,event,invited,timestamp
0,1776192,2877501688,0,2012-11-30 11:39:01.230000+00:00
1,1776192,3025444328,0,2012-11-30 11:39:01.230000+00:00
2,1776192,4078218285,0,2012-11-30 11:39:01.230000+00:00
3,1776192,1024025121,0,2012-11-30 11:39:01.230000+00:00
4,1776192,2972428928,0,2012-11-30 11:39:21.985000+00:00
5,1776192,2514143386,0,2012-11-30 11:39:01.230000+00:00
6,1776192,1823369186,0,2012-11-30 11:39:01.230000+00:00
7,5161061,2027962693,0,2012-11-07 16:20:11.484000+00:00
8,5161061,1652007005,0,2012-11-07 16:20:35.683000+00:00
9,5161061,2169802745,0,2012-11-07 16:20:32.153000+00:00


In [145]:
# test_dict = {user_id : {event_id : invited}}
def create_test_dict(test):
    duplicated_num = 0
    multiple_num = 0
    test_dict = {}
    for index, row in test.iterrows():
        user_id = str(row[0])
        event_id = str(row[1])
        invited = row[2]
        if not user_id in test_dict:
            test_dict[user_id] = {}
        if event_id in test_dict[user_id]:
            if test_dict[user_id][event_id] == invited:
                duplicated_num += 1
            else:
                multiple_num += 1
        else:
            test_dict[user_id][event_id] = invited
    print("Duplicated data in user: " + str(duplicated_num))
    print("Multiple data in user: " + str(multiple_num))
    return test_dict

In [11]:
user_friends.iloc[0 : 10,]

,user,friends
0,3197468391,1346449342 3873244116 4226080662 1222907620 54...
1,3537982273,1491560444 395798035 2036380346 899375619 3534...
2,823183725,1484954627 1950387873 1652977611 4185960823 42...
3,1872223848,83361640 723814682 557944478 1724049724 253059...
4,3429017717,4253303705 2130310957 1838389374 3928735761 71...
5,627175141,3462311094 868148671 3475458679 1822640148 183...
6,2752000443,665103859 798664587 3773945815 595192956 31571...
7,3473687777,1481860022 3611032568 3086070277 275090520 366...
8,2966052962,4266041793 2113918851 4152864981 1055028635 12...
9,264876277,1473289379 3127593523 487736094 2990183113 249...


In [12]:
# friends_dict = {user_id : []}
def create_friends_dict(user_friends):
    user_friends_dict = {}
    for index, row in user_friends.iterrows():
        user_id = str(row[0])
        if str(row[1]) == "nan":
            continue;
        else:
            friends_list = row[1].split(" ")
            user_friends_dict[user_id] = friends_list
    return user_friends_dict

In [13]:
users.iloc[0 : 10,]

,user_id,locale,birthyear,gender,joinedAt,location,timezone
0,3197468391,id_ID,1993,male,2012-10-02T06:40:55.524Z,Medan Indonesia,480.0
1,3537982273,id_ID,1992,male,2012-09-29T18:03:12.111Z,Medan Indonesia,420.0
2,823183725,en_US,1975,male,2012-10-06T03:14:07.149Z,Stratford Ontario,-240.0
3,1872223848,en_US,1991,female,2012-11-04T08:59:43.783Z,Tehran Iran,210.0
4,3429017717,id_ID,1995,female,2012-09-10T16:06:53.132Z,NaN,420.0
5,627175141,ka_GE,1973,female,2012-11-01T09:59:17.590Z,Tbilisi Georgia,240.0
6,2752000443,id_ID,1994,male,2012-10-03T05:22:17.637Z,Medan Indonesia,420.0
7,3473687777,id_ID,1965,female,2012-10-03T12:19:29.975Z,Medan Indonesia,420.0
8,2966052962,id_ID,1979,male,2012-10-31T10:11:57.668Z,Medan Indonesia,420.0
9,264876277,id_ID,1988,female,2012-10-02T07:28:09.555Z,Medan Indonesia,420.0


In [14]:
# users_dict = {user_id : [birthday, gender]}
def create_users_dict(users):
    user_dict = {}
    for index, row in users.iterrows():
        user_id = str(row[0])
        birthday = row[2]
        gender = row[3]
        user_dict[user_id] = [birthday, gender]
    return user_dict

In [15]:
events.iloc[0 : 10,]

,event_id,user_id,start_time,city,state,zip,country,lat,lng,c_1,...,c_92,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other
0,684921758,3647864012,2012-10-31T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,2,...,0,1,0,0,0,0,0,0,0,9
1,244999119,3476440521,2012-11-03T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,2,...,0,0,0,0,0,0,0,0,0,7
2,3928440935,517514445,2012-11-05T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,12
3,2582345152,781585781,2012-10-30T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,8
4,1051165850,1016098580,2012-09-27T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,9
5,1212611096,1426522332,2012-11-16T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,22
6,3689283674,725266702,2012-11-02T20:00:00.003Z,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,28
7,2584113432,613687941,2012-10-31T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,0,...,2,0,0,0,0,0,0,0,0,354
8,3365728297,1098509207,2012-10-31T00:00:00.001Z,NaN,NaN,NaN,NaN,47.058,21.926,0,...,0,0,0,0,0,0,0,1,0,25
9,2912638473,3598071768,2012-10-18T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,3


In [16]:
# events_dict = {event_id : [host_user_id, [c_list]]}
def create_events_dict(events):
    events_dict = {}
    for index, row in events.iterrows():
        event_id = str(row[0])
        host_id = str(row[1])
        c_list = list(row[9 : -1])
        events_dict[event_id] = [host_id, c_list]
    return events_dict

In [17]:
event_attendees_dict, user_interests_dict = create_event_attendees_dict_and_user_interests_dict(event_attendees)

In [148]:
train_dict = create_train_dict(train)

Duplicated data in user: 178
Multiple data in user: 0


In [149]:
test_dict = create_test_dict(test)

Duplicated data in user: 131
Multiple data in user: 0


In [20]:
friends_dict = create_friends_dict(user_friends)

In [21]:
users_dict = create_users_dict(users)

In [22]:
events_dict = create_events_dict(events)

In [130]:
# write train data with 214 features
f = open('train_data.csv','w')
thewriter = csv.writer(f)
title = [None] * 214
title[0] = "user_id"
title[1] = "event_id"
title[2] = "interested"
title[3] = "un_interested"
title[4] = "invited"
title[5] = "friend_yes"
title[6] = "friend_maybe"
title[7] = "friend_invited"
title[8] = "friend_no"
title[9] = "same_gender_yes"
title[10] = "same_gender_maybe"
title[11] = "same_gender_invited"
title[12] = "same_gender_no"
title[13] = "host_is_friend"
for i in range(14, 114):
    title[i] = "event_c_" + str(i - 14)
for i in range(114, 214):
    title[i] = "interest_c_" + str(i - 14)
thewriter.writerow(title)
    
train_data = []
count = 0
for user_id in train_dict.keys():
    event_inf_dict = train_dict[user_id]
    for event_id in event_inf_dict.keys():
        data = [None] * 214
        
        # feature 0 - 1: user_id, event id
        data[0] = user_id
        data[1] = event_id

        # feature 2 - 4: interested, un_interested, invited
        data[2] = event_inf_dict[event_id][1]
        data[3] = event_inf_dict[event_id][2]
        data[4] = event_inf_dict[event_id][0]

        friends_list = None
        attended_list = None
        user = None
        event = None
        interest_list = None
        if user_id in friends_dict:
            friends_list = friends_dict[user_id]
        if event_id in event_attendees_dict:
            attended_list = event_attendees_dict[event_id]
        if user_id in users_dict:
            user = users_dict[user_id]
        if event_id in events_dict:
            event = events_dict[event_id]
        if user_id in user_interests_dict:
            interest_list = user_interests_dict[user_id]

        # feature 5 - 8: friends attendency
        number_friends_yes = 0
        number_friends_maybe = 0
        number_friends_invited = 0
        number_friends_no = 0
        if friends_list != None and attended_list != None:
            for friend in friends_list:
                if friend in attended_list["1"]:
                    number_friends_yes += 1
                elif friend in attended_list["2"]:
                    number_friends_maybe += 1
                elif friend in attended_list["3"]:
                    number_friends_invited += 1
                elif friend in attended_list["4"]:
                    number_friends_no += 1
        data[5] = number_friends_yes
        data[6] = number_friends_maybe
        data[7] = number_friends_invited
        data[8] = number_friends_no

        # feature 9 - 12: same gender rate in event
        if user != None and attended_list != None:
            my_gender = user[1]
            for i in range(1, 5):
                same_gender = 0
                total = 0
                for guest_id in attended_list[str(i)]:
                    guest_gender = None
                    if guest_id in users_dict:
                        guest_gender = users_dict[guest_id][1]
                    if guest_gender == my_gender:
                        same_gender += 1
                    if guest_gender != None:
                        total += 1
                if total != 0:
                    data[i + 8] = same_gender / total
                else:
                    data[i + 8] = 0
        else:
            for i in range(9, 13):
                data[i] = 0

        # feature 13: host is friend:
        if event != None and friends_list != None:
            host_id = event[0]
            if host_id in friends_list:
                data[13] = 1
            else:
                data[13] = 0
        else:
            data[13] = 0

        # feature 14 - 113: c1 - c100
        if event != None:
            my_c = event[1]
            for i in range(0, 100):
                data[14 + i] = my_c[i]
        else:
            for i in range(14, 114):
                data[i] = 0
            
        # feature 114 - 213: my interets c1 - c100 mean
        if interest_list != None:
            if "1" in interest_list:
                my_interest_c_list = []
                for interest_id in interest_list["1"]:
                    if interest_id in events_dict:
                        my_interest_c_list.append(events_dict[interest_id][1])
                if len(my_interest_c_list) != 0:
                    mtx = np.matrix(my_interest_c_list)
                    mean = mtx.mean(0)
                    l = mean.tolist()[0]
                    for i in range(0, len(l)):
                        data[i + 114] = l[i]
                else:
                    for i in range(114, 214):
                        data[i] = 0
            else:
                for i in range(114, 214):
                    data[i] = 0
        else:
            for i in range(114, 214):
                data[i] = 0
        thewriter.writerow(data)
    
        
f.close()

KeyboardInterrupt: 

In [144]:
# write test data with 214 features
f2 = open('test_data.csv', 'w')
print("test_data.cvs file created")
print("data writing ...")
thewriter2 = csv.writer(f2)
title = [None] * 214
title[0] = "user_id"
title[1] = "event_id"
title[2] = "interested"
title[3] = "un_interested"
title[4] = "invited"
title[5] = "friend_yes"
title[6] = "friend_maybe"
title[7] = "friend_invited"
title[8] = "friend_no"
title[9] = "same_gender_yes"
title[10] = "same_gender_maybe"
title[11] = "same_gender_invited"
title[12] = "same_gender_no"
title[13] = "host_is_friend"
for i in range(14, 114):
    title[i] = "event_c_" + str(i - 14)
for i in range(114, 214):
    title[i] = "interest_c_" + str(i - 14)
thewriter2.writerow(title)

test_data = []
count = 0
for user_id in test_dict.keys():
    event_inf_dict = test_dict[user_id]
    for event_id in event_inf_dict.keys():
        data = [None] * 214

        # feature 0 - 1: user_id, event id
        data[0] = user_id
        data[1] = event_id

        # feature 2 - 4: interested, un_interested, invited
        data[2] = -1
        data[3] = -1
        data[4] = event_inf_dict[event_id]

        friends_list = None
        attended_list = None
        user = None
        event = None
        interest_list = None
        if user_id in friends_dict:
            friends_list = friends_dict[user_id]
        if event_id in event_attendees_dict:
            attended_list = event_attendees_dict[event_id]
        if user_id in users_dict:
            user = users_dict[user_id]
        if event_id in events_dict:
            event = events_dict[event_id]
        if user_id in user_interests_dict:
            interest_list = user_interests_dict[user_id]

        # feature 5 - 8: friends attendency
        number_friends_yes = 0
        number_friends_maybe = 0
        number_friends_invited = 0
        number_friends_no = 0
        if friends_list != None and attended_list != None:
            for friend in friends_list:
                if friend in attended_list["1"]:
                    number_friends_yes += 1
                elif friend in attended_list["2"]:
                    number_friends_maybe += 1
                elif friend in attended_list["3"]:
                    number_friends_invited += 1
                elif friend in attended_list["4"]:
                    number_friends_no += 1
        data[5] = number_friends_yes
        data[6] = number_friends_maybe
        data[7] = number_friends_invited
        data[8] = number_friends_no

        # feature 9 - 12: same gender rate in event
        if user != None and attended_list != None:
            my_gender = user[1]
            for i in range(1, 5):
                same_gender = 0
                total = 0
                for guest_id in attended_list[str(i)]:
                    guest_gender = None
                    if guest_id in users_dict:
                        guest_gender = users_dict[guest_id][1]
                    if guest_gender == my_gender:
                        same_gender += 1
                    if guest_gender != None:
                        total += 1
                if total != 0:
                    data[i + 8] = same_gender / total
                else:
                    data[i + 8] = 0
        else:
            for i in range(9, 13):
                data[i] = 0

        # feature 13: host is friend:
        if event != None and friends_list != None:
            host_id = event[0]
            if host_id in friends_list:
                data[13] = 1
            else:
                data[13] = 0
        else:
            data[13] = 0

        # feature 14 - 113: c1 - c100
        if event != None:
            my_c = event[1]
            for i in range(0, 100):
                data[14 + i] = my_c[i]
        else:
            for i in range(14, 114):
                data[i] = 0

        # feature 114 - 213: my interets c1 - c100 mean
        if interest_list != None:
            if "1" in interest_list:
                my_interest_c_list = []
                for interest_id in interest_list["1"]:
                    if interest_id in events_dict:
                        my_interest_c_list.append(events_dict[interest_id][1])
                if len(my_interest_c_list) != 0:
                    mtx = np.matrix(my_interest_c_list)
                    mean = mtx.mean(0)
                    l = mean.tolist()[0]
                    for i in range(0, len(l)):
                        data[i + 114] = l[i]
                else:
                    for i in range(114, 214):
                        data[i] = 0
            else:
                for i in range(114, 214):
                    data[i] = 0
        else:
            for i in range(114, 214):
                data[i] = 0
        thewriter2.writerow(data)

f2.close()
print("finished!!!!")

test_data.cvs file created
data writing ...
['1776192', '1823369186', -1, -1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 4, 1, 0, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['5161061', '2098674667', -1, -1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 3, 2, 1, 0, 0, 0, 1, 0, 1, 2, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 2, 1, 1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 

['38395056', '3415281246', -1, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 3, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['43826519', '2378630297', -1, -1, 0, 1, 0, 7, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

['76313799', '1469291456', -1, -1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['77627621', '2429151401', -1, -1, 0, 0, 0, 2, 0, 0, 0, 0.75, 1.0, 0, 1, 1, 1, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0

['110128780', '955398943', -1, -1, 0, 1, 0, 3, 0, 0.7586206896551724, 0.6875, 0.42857142857142855, 1.0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['110342379', '642644408', -1, -1, 0, 0, 0, 3, 0, 0.8, 0, 0.5, 0.0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

['147627126', '2698678033', -1, -1, 0, 1, 0, 11, 0, 0, 0, 0.0, 0, 0, 1, 0, 0, 0, 2, 47, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['149914134', '1076364848', -1, -1, 0, 6, 4, 51, 4, 0.5555555555555556, 0.37037037037037035, 0.46875, 0.5555555555555556, 0, 14, 7, 9, 4, 5, 3, 3, 5, 2, 2, 6, 6, 2, 3, 3, 1, 2, 7, 4, 1, 1, 2, 0, 0, 5, 5, 1, 2, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 1, 

...

['489332113', '2536770547', -1, -1, 0, 0, 0, 3, 1, 0, 1.0, 0.5652173913043478, 0.5, 0, 2, 4, 3, 5, 5, 0, 1, 3, 1, 0, 0, 0, 3, 2, 1, 3, 3, 2, 2, 1, 0, 2, 1, 1, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 4, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 13.0, 7.0, 9.0, 3.0, 8.5, 3.0, 3.0, 4.0, 4.0, 2.0, 3.5, 3.0, 2.0, 4.0, 2.0, 2.0, 1.5, 2.5, 1.5, 1.5, 0.0, 1.0, 1.0, 1.0, 2.0, 0.0, 1.0, 1.0, 1.5, 0.0, 0.0, 0.0, 1.5, 0.5, 0.5, 0.5, 2.0, 0.0, 1.0, 3.5, 0.5, 0.0, 0.5, 0.0, 0.5, 0.0, 0.5, 0.0, 1.5, 0.0, 0.0, 1.5, 0.0, 1.5, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.5, 1.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.5, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 1.0, 0.5, 1.0, 1.0, 0.0, 0.5, 1.5, 0.5, 0.5, 0.0, 0.5, 0.5, 1.5, 0.5, 0.0, 0.5]
['491496715', '244220291', -1, -1, 0, 0, 0, 1, 0, 0.6666666666666666, 0, 1.0, 0.0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,

['516201949', '3501509607', -1, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['521412534', '1269035551', -1, -1, 0, 0, 0, 9, 0, 0.25, 0.45454545454545453, 0.41911764705882354, 0, 0, 2, 1, 3, 0, 0, 0, 2, 1, 3, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 0, 0, 2, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0

['554693050', '1116367368', -1, -1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['555727671', '1390707377', -1, -1, 0, 1, 1, 87, 0, 0.3333333333333333, 0.5, 0.3157894736842105, 0.0, 0, 2, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

['589386328', '4156197678', -1, -1, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['589868615', '2678142390', -1, -1, 0, 0, 1, 13, 1, 0, 0, 0.6, 0, 0, 9, 14, 0, 7, 6, 0, 0, 1, 2, 4, 1, 1, 2, 2, 1, 1, 1, 0, 0, 1, 4, 3, 0, 0, 1, 0, 1, 3, 0, 0, 2, 0, 2, 0, 1, 2, 1, 0, 0, 0, 1, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 

KeyboardInterrupt: 

In [139]:
user_interests_dict["33509612"]

KeyError: '33509612'